### 8. 데이터 준비하기 : 조인, 병합, 변형
#### 8.1 계층적 색인
- 차원수가 높은 데이터를 낮은 차원의 형식으로 다룰 수 있게 해주는 기능

In [1]:
# 리스트를 담고있는 리스트를 색인으로 가진 Series
import pandas as pd
import numpy as np
data = pd.Series(np.random.uniform(size=9),
                index = [["a","a","a","b","b","c","c","d","d"], [1,2,3,1,3,1,2,2,3]])
data

C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\lockd\anaconda3\envs\pyTest\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


a  1    0.249366
   2    0.488458
   3    0.861045
b  1    0.889852
   3    0.657551
c  1    0.709750
   2    0.113714
d  2    0.897307
   3    0.821940
dtype: float64

In [2]:
# 바로 위 단계의 색인을 이용해서 하위 계층에 직접 접근할 수 있다
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [3]:
# 계층적으로 색인된 객체는 데이터의 부분집합을 부분적 색인으로 접근하는것이 가능
data["b"]

1    0.889852
3    0.657551
dtype: float64

In [4]:
data["b":"c"]

b  1    0.889852
   3    0.657551
c  1    0.709750
   2    0.113714
dtype: float64

In [5]:
data.loc[["b","d"]]

b  1    0.889852
   3    0.657551
d  2    0.897307
   3    0.821940
dtype: float64

In [6]:
# 하위계층의 객체를 선택하는 것도 가능
data.loc[:,2] #두번째 단계 색인이 2인 모든 값을 선택

a    0.488458
c    0.113714
d    0.897307
dtype: float64

In [7]:
# 계층적 색인은 데이터를 재구성하고 피벗 테이블 생성 같은 그룹 기반으로 작업할 때 중요하게 사용된다
# unstack 메서드를 사용해 데이터를 새롭게 배열
data.unstack()

,1,2,3
a,0.249366,0.488458,0.861045
b,0.889852,NaN,0.657551
c,0.709750,0.113714,NaN
d,NaN,0.897307,0.821940


In [8]:
# stack메서드로 반대될 수 있음
data.unstack().stack()

a  1    0.249366
   2    0.488458
   3    0.861045
b  1    0.889852
   3    0.657551
c  1    0.709750
   2    0.113714
d  2    0.897307
   3    0.821940
dtype: float64

In [15]:
# DataFrame은 두 축 모두 계층적 색인을 가질 수 있음
import pandas as pd
import numpy as np
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index = [["a","a","b","b"],[1,2,1,2]],
                    columns = [["Ohio","Ohio","Colorado"],["Green","Red","Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [21]:
frame.loc['a'][1:2]

Ohio     Colorado
  Green Red    Green
2     3   4        5

In [17]:
frame.loc[('a',2)]

Ohio      Green    3
          Red      4
Colorado  Green    5
Name: (a, 2), dtype: int32

In [16]:
frame.loc[('a',2)].dtype

dtype('int32')

In [15]:
frame.index.names = ["key1","key2"]
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [16]:
# 여기서 사용된 이름은 단일 색인에서만 사용되는 name 속성을 대체
# 색인 이름인 state.color를 행 레이블(frame.index)과 혼동하지말자
# nlevels 속성으로 확인
frame.index.nlevels

2

In [17]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [18]:
# 멀티 인덱스는 재사용 가능
pd.MultiIndex.from_arrays([["Ohio","Ohio","Colorado"],
                          ["Green","Red","Green"]],
                         names=["state","color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

#### 1) 계층의 순서를 바꾸고 정렬하기
- swaplevel은 넘겨받은 두 개의 계층이 번호나 이름이 뒤바뀐 새로운 객체를 반환

In [19]:
frame.swaplevel("key1","key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [20]:
# sort_index 메서드는 모든 색인 계층을 사용해 사전 순으로 데이터 정렬
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [21]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

#### 2) 계층별 요약 통계
- 행이나 열의 합을 계층별로 구할 수 있다

In [24]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [25]:
frame.groupby(level="color", axis="columns").sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

#### 3) DataFrame의 열 사용하기

In [26]:
frame = pd.DataFrame({"a":range(7), "b":range(7,0,-1),
                     "c":["one","one","one","two","two","two","two"],
                     "d":[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [27]:
frame2 = frame.set_index(["c","d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [28]:
# set_index 메서드에 drop=False를 명시적으로 지정하지 않으면 색인으로 지정한 열은 DataFrame 에서 삭제된다
frame.set_index(["c","d"],drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [29]:
# set_index와 반대개념. 색인단계가 열로 이동
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### 8.2. 데이터 합치기
- pandas.merge : 하나 이상의 키를 기준으로 DataFrame의 행을 연결. SQL이나 다른 관계형 데이터베이스의 조인 연산과 유사
- pandas.concat : 하나의 축을 따라 객체를 이어붙임
- combine_first : 두 객체를 겹쳐서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채움

#### 1) 데이터베이스 스타일로 DataFrame 합치기

In [30]:
df1 = pd.DataFrame({"key":["b","b","a","c","a","a","b"],
                   "data1":pd.Series(range(7), dtype="Int64")})

In [31]:
df2 = pd.DataFrame({"key":["a","b","d"],
                   "data2":pd.Series(range(3), dtype="Int64")})

In [32]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [33]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [34]:
# df1 데이터는 key 열에 여러개의 a,b를 가지며 df2의 key열은 유일한 행을 갖는다
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [35]:
pd.merge(df1,df2) #명시하지 않아도 겹치는 열의 이름을 키로 사용

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [36]:
pd.merge(df1,df2,on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [37]:
# 공통되는 열이름이 없다면 별도지정
df3 = pd.DataFrame({"lkey":["b","b","a","c","a","a","b"],
                   "data1":pd.Series(range(7), dtype="Int64")})

In [38]:
df4 = pd.DataFrame({"rkey":["a","b","d"],
                   "data2":pd.Series(range(3),dtype="Int64")})

In [39]:
pd.merge(df3, df4, left_on="lkey",right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [40]:
#merge함수의 how에 left, right, outer을 넘겨 조인방식을 설정할 수 있다. 
# outer > 합집합결과,left&right > 왼쪽 또는 오른쪽 모든행을 포함하는 결과를 반환
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0
6,c,3,<NA>
7,d,<NA>,2


In [41]:
pd.merge(df3, df4, left_on="lkey",right_on="rkey",how="outer")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [3]:
df1 = pd.DataFrame({"key":["b","b","a","c","a","b"],
                   "data1":pd.Series(range(6), dtype="Int64")})

In [4]:
df2 = pd.DataFrame({"key":["a","b","a","b","d"],
                   "data2":pd.Series(range(5), dtype="Int64")})

In [5]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [6]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [7]:
pd.merge(df1, df2, on="key",how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [8]:
import pandas as pd
pd.merge(df1, df2, how="inner") #교집합
# how인수는 결과에 나타나는 구별되는 키에 대해서만 적용

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [11]:
left = pd.DataFrame({"key1":["foo","foo","bar"],
                    "key2" : ["one","two","one"],
                    "lval" : pd.Series([1,2,3],dtype='Int64')})

In [12]:
right = pd.DataFrame({"key1": ["foo","foo","bar","bar"],
                     "key2":["one","one","one","two"],
                     "rval":pd.Series([4,5,6,7], dtype="Int64")})

In [13]:
pd.merge(left,right,on=["key1","key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,<NA>
3,bar,one,3,6
4,bar,two,<NA>,7


In [14]:
#겹치는 열 이름 처리 방식
pd.merge(left,right,on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [15]:
pd.merge(left,right,on="key1", suffixes=("_left","_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


#### 2) 색인 병합하기

In [16]:
left1 = pd.DataFrame({"key":["a","b","a","a","b","c"],
                             "value":pd.Series(range(6), dtype="Int64")})

In [17]:
right1 = pd.DataFrame({"group_val":[3.5,7]}, index=["a","b"])

In [18]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [19]:
right1

,group_val
a,3.5
b,7.0


In [21]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [22]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [23]:
lefth = pd.DataFrame({"key1":["Ohio","Ohio","Ohio",
                             "Nevada","Nevada"],
                     "key2":[2000, 2001, 2002, 2001, 2002],
                     "data" : pd.Series(range(5), dtype="Int64")})

In [24]:
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada","Nevada","Ohio","Ohio","Ohio","Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)

In [25]:
righth = pd.DataFrame({"event1":pd.Series([0,2,4,6,8,10], dtype="Int64", index=righth_index),
                      "event2":pd.Series([1,3,5,7,9,11], dtype="Int64", index=righth_index)})

In [26]:
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [27]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [28]:
# 이때는 리스트로 여러 개의 열을 지정해서 병합해야한다
pd.merge(lefth, righth, left_on=["key1","key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [30]:
pd.merge(lefth, righth, left_on=["key1","key2"],
        right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


In [25]:
left2 = pd.DataFrame([[1.,2.],[3.,4.],[5.,6.]],
                    index = ["a","c","e"],
                    columns = ["Ohio","Nevada"]).astype("Int64")

In [26]:
right2 = pd.DataFrame([[7.,8.], [9.,10,],[11.,12.], [13,14]],
                     index = ["b","c","d","e"],
                     columns=["Missouri","Alabame"]).astype("Int64")

In [27]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [28]:
right2

,Missouri,Alabame
b,7,8
c,9,10
d,11,12
e,13,14


In [29]:
pd.merge(left2, right2, how="outer", left_index = True, right_index=True)

,Ohio,Nevada,Missouri,Alabame
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [30]:
#색인으로 병합할 때 join 메서드를 사용하면 편리하다
#기본적으로 왼쪽 조인 수행
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabame
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [31]:
left1.join(right1, on="key")

NameError: name 'left1' is not defined

In [22]:
another = pd.DataFrame([[7.,8.], [9.,10.], [11.,12.],[16.,17.]],
                      index = ["a","c","e","f"],
                      columns = ["New York","Oregon"])

In [23]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [24]:
left2.join([right2, another])

NameError: name 'left2' is not defined

In [46]:
#색인대 색인으로 두 DataFame을 합친다
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabame,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


#### 3) 축 따라 이어붙이기 - concat

In [50]:
import numpy as np
arr = np.arange(12).reshape((3,4))

In [51]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [52]:
np.concatenate([arr, arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

- 판다스 객체의 컨텍스트 내부에는 축마다 이름이 있어 배열을 쉽게 이어 붙일 수 있음
- 고려사항
  - 두 객체의 색인이 서로 다르면 교집합 or 합집합?
  - 합쳐진 결과에서 합쳐지기 전 객체의 데이터를 구분할 수 있어야하는가?
  - 어떤 축으로 연결?
  
 => concat으로 해결

Merge는 눈치있게 알아서 value를 기준으로 합쳐주는데,

Join은 index를 기준으로하는거라, 데이터를 합치기전에 index를 세팅해줘야된다. 

In [53]:
s1 = pd.Series([0,1], index = ["a","b"], dtype="Int64")
s2 = pd.Series([2,3,4], index = ["c","d","e"], dtype="Int64")
s3 = pd.Series([5,6], index = ["f","g"], dtype="Int64")

In [54]:
s1

a    0
b    1
dtype: Int64

In [55]:
s2

c    2
d    3
e    4
dtype: Int64

In [56]:
s3

f    5
g    6
dtype: Int64

In [57]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [58]:
#axis = index를 기본값으로 하고 새로운 Series 객체 생성. axis = "columns" 면 DataFrame 생성
pd.concat([s1,s2,s3], axis = "columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [59]:
s4 = pd.concat([s1,s3])

In [60]:
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [61]:
pd.concat([s1, s4], axis = "columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [62]:
pd.concat([s1, s4],axis="columns",join="inner")

,0,1
a,0,0
b,1,1


In [63]:
#계층적  색인을 생성하려면 keys 인수를 사용
result = pd.concat([s1,s1,s3],keys=["one","two","three"])

In [64]:
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [65]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


In [67]:
pd.concat([s1,s2,s3], axis="columns",keys=["one","two","three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6
